In [3]:
import numpy as np

class NeuralNetwork():

    # Set random seed for reproducibility
    np.random.seed(42)

    def __init__(self, X, y, n_hidden_neurons, output_act_fn='sigmoid', error_fn='cross_entropy'):
        self.X = X
        self.y = y
        self.n_input_neurons = X.shape[1]
        self.n_hidden_neurons = n_hidden_neurons
        self.output_act_fn = output_act_fn
        self.error_fn = error_fn

        # He initialization for stability
        self.input_hidden_weights = np.random.randn(self.n_input_neurons, self.n_hidden_neurons) * np.sqrt(2 / self.n_input_neurons)
        self.hidden_biases = np.zeros(self.n_hidden_neurons)
        self.hidden_output_weights = np.random.randn(self.n_hidden_neurons, 1) * np.sqrt(2 / self.n_hidden_neurons)
        self.output_bias = np.zeros(1)

    # ---------------- Activation Functions ----------------
    def activation(self, x, act_fn):
        if act_fn == 'sigmoid':
            x = np.clip(x, -500, 500)  # Prevent overflow
            return 1 / (1 + np.exp(-x))
        elif act_fn == 'relu':
            return np.maximum(0, x)
        elif act_fn == 'linear':
            return x
        else:
            raise Exception('Unknown activation function')

    def activation_derivative(self, x, act_fn):
        if act_fn == 'sigmoid':
            return x * (1 - x)  # x here is the sigmoid output
        elif act_fn == 'relu':
            return np.where(x > 0, 1, 0)
        elif act_fn == 'linear':
            return 1
        else:
            raise Exception('Unknown activation function')

    # ---------------- Forward Propagation ----------------
    def forward_pass(self, X):
        # Hidden layer
        self.z_hidden = np.dot(X, self.input_hidden_weights) + self.hidden_biases
        self.hidden = self.activation(self.z_hidden, 'relu')

        # Output layer
        self.z_output = np.dot(self.hidden, self.hidden_output_weights) + self.output_bias
        self.output = self.activation(self.z_output, self.output_act_fn)

        return self.output

    # ---------------- Error Computation ----------------
    def error(self, y_true, y_pred):
        if self.error_fn == 'mse':
            return np.mean(np.square(y_true - y_pred))
        elif self.error_fn == 'cross_entropy':
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
            return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        else:
            raise Exception('Unknown error function')

    def error_derivative(self, y_true, y_pred):
        # Simplified stable gradient for cross-entropy
        if self.error_fn == 'mse':
            return 2 * (y_pred - y_true) / y_true.size
        elif self.error_fn == 'cross_entropy':
            return (y_pred - y_true) / y_true.size
        else:
            raise Exception('Unknown error function')

    # ---------------- Backward Propagation ----------------
    def backward_pass(self, X, y_true, y_pred, learning_rate):
        # Output layer
        self.output_error = self.error_derivative(y_true, y_pred) * self.activation_derivative(y_pred, self.output_act_fn)
        self.hidden_output_weights -= learning_rate * np.dot(self.hidden.T, self.output_error)
        self.output_bias -= learning_rate * np.sum(self.output_error, axis=0)

        # Hidden layer
        self.hidden_error = np.dot(self.output_error, self.hidden_output_weights.T) * self.activation_derivative(self.hidden, 'relu')
        self.input_hidden_weights -= learning_rate * np.dot(X.T, self.hidden_error)
        self.hidden_biases -= learning_rate * np.sum(self.hidden_error, axis=0)

        return self.input_hidden_weights, self.hidden_biases, self.hidden_output_weights, self.output_bias

    # ---------------- Training ----------------
    def train(self, X, y, learning_rate=0.001, epochs=5000):
        for epoch in range(epochs):
            y_pred = self.forward_pass(X)
            wih, bh, who, bo = self.backward_pass(X, y, y_pred, learning_rate)
            if epoch % 500 == 0:
                print(f'Epoch: {epoch}, Loss: {self.error(y, y_pred):.4f}')
        print('✅ Training complete!')
        return wih, bh, who, bo

    # ---------------- Prediction ----------------
    def predict(self, X):
        preds = self.forward_pass(X)
        if self.error_fn == 'mse':
            return preds
        elif self.error_fn == 'cross_entropy':
            return np.where(preds > 0.5, 1, 0)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

# Generate a regression dataset
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1 , random_state=42)
y = y.reshape(-1, 1)  # Reshape y to be a 2D array

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.preprocessing import StandardScaler
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Initialize and train the neural network
nn = NeuralNetwork(X_train, y_train, n_hidden_neurons=128, output_act_fn='linear', error_fn='mse')
wih , bh , who , bo = nn.train(X_train, y_train, learning_rate=0.001, epochs=5000 )    

Epoch: 0, Loss: 17433.7012
Epoch: 500, Loss: 3.8558
Epoch: 1000, Loss: 1.8428
Epoch: 1500, Loss: 1.2564
Epoch: 2000, Loss: 0.9450
Epoch: 2500, Loss: 0.7630
Epoch: 3000, Loss: 0.6403
Epoch: 3500, Loss: 0.5501
Epoch: 4000, Loss: 0.4783
Epoch: 4500, Loss: 0.4242
✅ Training complete!


In [6]:
from sklearn.metrics import r2_score, mean_squared_error
# Make predictions on the test set  
y_pred = nn.predict(X_test)
# Evaluate the model
print(f'R^2 Score: {r2_score(y_test, y_pred):.4f}')
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}')
# ---------------- Evaluation ----------------
for i in range(len(y_test)):
    print(f'Actual: {y_test[i][0]:.2f}, Predicted: {y_pred[i][0]:.2f}')


R^2 Score: 0.9999
Mean Squared Error: 2.2291
Actual: 42.67, Predicted: 43.69
Actual: 75.01, Predicted: 73.48
Actual: -4.06, Predicted: -4.73
Actual: -295.72, Predicted: -296.71
Actual: 44.43, Predicted: 44.72
Actual: 21.68, Predicted: 20.70
Actual: -146.52, Predicted: -145.94
Actual: -60.56, Predicted: -59.54
Actual: -15.32, Predicted: -13.44
Actual: 176.65, Predicted: 178.07
Actual: -325.84, Predicted: -325.62
Actual: -56.63, Predicted: -57.26
Actual: 46.26, Predicted: 45.16
Actual: -88.99, Predicted: -89.48
Actual: 127.20, Predicted: 126.43
Actual: -134.09, Predicted: -132.58
Actual: -63.02, Predicted: -62.61
Actual: 47.02, Predicted: 47.18
Actual: 92.68, Predicted: 94.89
Actual: -40.77, Predicted: -41.47
Actual: 128.26, Predicted: 127.64
Actual: 239.29, Predicted: 237.24
Actual: 191.73, Predicted: 193.44
Actual: -129.84, Predicted: -130.68
Actual: 184.19, Predicted: 183.33
Actual: -123.97, Predicted: -125.10
Actual: -247.00, Predicted: -247.02
Actual: 250.06, Predicted: 249.37
Actua

In [7]:
from sklearn.linear_model import LinearRegression
# Train a linear regression model for comparison
lr = LinearRegression()
lr.fit(X_train, y_train)
y_lr_pred = lr.predict(X_test)
# Evaluate the linear regression model
print(f'Linear Regression R^2 Score: {r2_score(y_test, y_lr_pred):.4f}')
print(f'Linear Regression Mean Squared Error: {mean_squared_error(y_test, y_lr_pred):.4f}')

Linear Regression R^2 Score: 1.0000
Linear Regression Mean Squared Error: 0.0095


In [8]:
from sklearn.neural_network import MLPRegressor
# Train a built-in MLP regressor for comparison
mlp = MLPRegressor(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=5000, random_state=42)
mlp.fit(X_train, y_train.ravel())
y_mlp_pred = mlp.predict(X_test)
# Evaluate the MLP regressor    
print(f'MLP Regressor R^2 Score: {r2_score(y_test, y_mlp_pred):.4f}')
print(f'MLP Regressor Mean Squared Error: {mean_squared_error(y_test, y_mlp_pred):.4f}')


MLP Regressor R^2 Score: 0.9999
MLP Regressor Mean Squared Error: 2.1758


In [10]:
from sklearn.datasets import make_classification
# Generate a classification dataset
X_clf, y_clf = make_classification(n_samples=1000, n_features=10, n_classes=2, n_informative=5, random_state=42)
y_clf = y_clf.reshape(-1, 1)  # Reshape y to be a 2D array
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)
X_clf_train = scaler.fit_transform(X_clf_train)
X_clf_test = scaler.transform(X_clf_test)
# Initialize and train the neural network for classification
nn_clf = NeuralNetwork(X_clf_train, y_clf_train, n_hidden_neurons=128, output_act_fn='sigmoid', error_fn='cross_entropy')
wih_clf , bh_clf , who_clf , bo_clf = nn_clf.train(X_clf_train, y_clf_train, learning_rate=0.001, epochs=5000 ) 

Epoch: 0, Loss: 0.8644
Epoch: 500, Loss: 0.6629
Epoch: 1000, Loss: 0.5848
Epoch: 1500, Loss: 0.5432
Epoch: 2000, Loss: 0.5141
Epoch: 2500, Loss: 0.4915
Epoch: 3000, Loss: 0.4734
Epoch: 3500, Loss: 0.4585
Epoch: 4000, Loss: 0.4461
Epoch: 4500, Loss: 0.4356
✅ Training complete!


In [11]:
print("Neural Network Classification Results:" )
y_clf_pred = nn_clf.predict(X_clf_test)
accuracy = np.mean(y_clf_pred == y_clf_test)
print(f'Accuracy: {accuracy:.4f}')

Neural Network Classification Results:
Accuracy: 0.8100


In [12]:
from sklearn.neural_network import MLPClassifier
# Train a built-in MLP classifier for comparison
mlp_clf = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver ='adam', max_iter=5000, random_state=42)
mlp_clf.fit(X_clf_train, y_clf_train.ravel())
y_mlp_clf_pred = mlp_clf.predict(X_clf_test)
mlp_accuracy = np.mean(y_mlp_clf_pred == y_clf_test.ravel())
print(f'MLP Classifier Accuracy: {mlp_accuracy:.4f}')

MLP Classifier Accuracy: 0.9500
